<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.4%20Modelado%20Secuencial/4.4.4%20An%C3%A1lisis%20de%20sentimientos%20con%20RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
# Se definen 10 comentarios de algún restaurante
reviews = [
          'Never coming back!',
          'horrible service',
          'rude waitress',
          'cold food',
          'horrible food!',
          'awesome',
          'awesome service!',
          'rocks',
          'nice work',
          'couldn\'t have done better'
]

# Se definen sus etiquetas. 1 para positivo, 0 para negativo
labels = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

#### Tokenización

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 50
oov_tok = '<OOV>'
padding_type = 'post'

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(reviews)
word_index = tokenizer.word_index

In [ ]:
word_index

{'<OOV>': 1,
 'horrible': 2,
 'service': 3,
 'food': 4,
 'awesome': 5,
 'never': 6,
 'coming': 7,
 'back': 8,
 'rude': 9,
 'waitress': 10,
 'cold': 11,
 'rocks': 12,
 'nice': 13,
 'work': 14,
 "couldn't": 15,
 'have': 16,
 'done': 17,
 'better': 18}

In [ ]:
tokenizer.word_counts

OrderedDict([('never', 1),
             ('coming', 1),
             ('back', 1),
             ('horrible', 2),
             ('service', 2),
             ('rude', 1),
             ('waitress', 1),
             ('cold', 1),
             ('food', 2),
             ('awesome', 2),
             ('rocks', 1),
             ('nice', 1),
             ('work', 1),
             ("couldn't", 1),
             ('have', 1),
             ('done', 1),
             ('better', 1)])

In [ ]:
vocab_size = len(word_index)
vocab_size

18

In [ ]:
tokenized_reviews = tokenizer.texts_to_sequences(reviews)
tokenized_reviews

[[6, 7, 8],
 [2, 3],
 [9, 10],
 [11, 4],
 [2, 4],
 [5],
 [5, 3],
 [12],
 [13, 14],
 [15, 16, 17, 18]]

In [ ]:
max_length = max([len(x) for x in tokenized_reviews])
max_length

4

In [ ]:
padded_reviews = pad_sequences(tokenized_reviews, maxlen=max_length, padding=padding_type)
print(padded_reviews)

[[ 6  7  8  0]
 [ 2  3  0  0]
 [ 9 10  0  0]
 [11  4  0  0]
 [ 2  4  0  0]
 [ 5  0  0  0]
 [ 5  3  0  0]
 [12  0  0  0]
 [13 14  0  0]
 [15 16 17 18]]


#### Word Embeddings

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

In [ ]:
embedding_layer = Embedding(input_dim=vocab_size+1, output_dim=8, input_length=max_length)

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              152       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 185
Trainable params: 185
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 0.6899 - accuracy: 0.7000
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6878 - accuracy: 0.7000
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6858 - accuracy: 0.7000
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6838 - accuracy: 0.8000
Epoch 5/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6818 - accuracy: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6798 - accuracy: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6778 - accuracy: 0.9000
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6758 - accuracy: 0.9000
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6738 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6718 - accuracy: 1.0000
Epoch 11/100
1/1 [===

In [ ]:
import pandas as pd
from scipy import spatial

In [ ]:
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

In [ ]:
print(embedding_layer.get_weights()[0].shape)

(19, 8)


In [ ]:
pd.DataFrame((embedding_layer.get_weights()[0]), index=['<pad>']+list(word_index.keys()), columns=['dim_'+str(x) for x in range(1,9)])

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8
<pad>,-0.002872,-0.169136,0.081623,0.050798,-0.003134,0.113033,0.146987,-0.095296
<OOV>,-0.000066,-0.019284,0.020890,0.005598,0.031121,-0.022968,-0.030236,-0.049565
horrible,0.093230,0.092167,0.108238,0.067024,-0.152640,0.151768,-0.101885,-0.133817
service,-0.068107,0.022881,0.070349,0.020305,-0.017185,0.022792,-0.018833,-0.002838
food,0.118229,0.133906,-0.097039,-0.057849,0.146062,-0.145414,-0.143044,0.096243
awesome,-0.132440,-0.137892,-0.095382,-0.113152,0.090876,-0.093992,0.087166,0.074411
never,0.065052,0.084699,0.137119,0.100565,-0.079874,0.117911,-0.141945,-0.058283
coming,0.164593,0.114664,-0.088531,-0.126124,0.089613,-0.077185,-0.107477,0.121547
back,0.039234,0.046961,-0.052296,-0.100803,0.106401,-0.085425,0.124592,-0.148585
rude,0.099487,0.075202,0.102889,0.080231,-0.100884,0.125737,-0.147084,-0.146315


In [ ]:
computed_similarities = pd.DataFrame()

for word1 in word_index.keys():
    for word2 in word_index.keys():
        idx1 = word_index[word1]
        idx2 = word_index[word2]
        embedding1 = embedding_layer.get_weights()[0][idx1]
        embedding2 = embedding_layer.get_weights()[0][idx2]
        similarity = cosine_similarity(embedding1, embedding2)
        computed_similarities.loc[word1, word2] = similarity

In [ ]:
def non_zero_green(val):
    return 'background-color: Aquamarine' if (val > 0.9) & (val < 1) else ''

In [ ]:
computed_similarities.style.applymap(non_zero_green)

,,horrible,service,food,awesome,never,coming,back,rude,waitress,cold,rocks,nice,work,couldn't,have,done,better
,1.000000,0.093842,0.097693,0.096775,-0.057557,0.172329,-0.122584,0.309258,0.282958,0.076440,0.287669,-0.187660,-0.171752,0.101769,-0.142603,-0.067105,-0.072005,-0.349253
horrible,0.093842,1.000000,0.368913,-0.299805,-0.918938,0.920952,-0.215094,-0.298124,0.971423,-0.220611,0.893650,-0.925804,-0.955010,0.300891,-0.892946,0.261472,0.414309,-0.291866
service,0.097693,0.368913,1.000000,-0.443290,-0.270052,0.512904,-0.543753,-0.446043,0.336330,-0.629144,0.342862,-0.228076,-0.377663,0.626213,-0.442597,0.485251,0.502817,0.256386
food,0.096775,-0.299805,-0.443290,1.000000,0.032541,-0.143269,0.924630,0.191524,-0.173021,0.957440,-0.067101,0.015660,0.111445,-0.940567,0.177117,-0.972179,-0.396015,0.345249
awesome,-0.057557,-0.918938,-0.270052,0.032541,1.000000,-0.923799,0.001873,0.276487,-0.917909,-0.028322,-0.951443,0.974223,0.953388,-0.063291,0.959726,-0.023241,-0.315897,0.321069
never,0.172329,0.920952,0.512904,-0.143269,-0.923799,1.000000,-0.133333,-0.490091,0.944207,-0.131616,0.892905,-0.924831,-0.938234,0.198352,-0.927711,0.163839,0.545178,-0.215339
coming,-0.122584,-0.215094,-0.543753,0.924630,0.001873,-0.133333,1.000000,0.159035,-0.145241,0.935544,-0.077643,0.004250,0.120797,-0.978583,0.215176,-0.896648,-0.407587,0.350616
back,0.309258,-0.298124,-0.446043,0.191524,0.276487,-0.490091,0.159035,1.000000,-0.280029,0.260863,-0.094277,0.230264,0.290410,-0.132360,0.283380,-0.276498,-0.924177,-0.202124
rude,0.282958,0.971423,0.336330,-0.173021,-0.917909,0.944207,-0.145241,-0.280029,1.000000,-0.098252,0.913850,-0.958359,-0.967804,0.210164,-0.898949,0.145208,0.407994,-0.332885
waitress,0.076440,-0.220611,-0.629144,0.957440,-0.028322,-0.131616,0.935544,0.260863,-0.098252,1.000000,-0.032956,-0.072016,0.065597,-0.965884,0.142834,-0.959771,-0.454202,0.155336


#### Predicción

In [ ]:
reviews

['Never coming back!',
 'horrible service',
 'rude waitress',
 'cold food',
 'horrible food!',
 'awesome',
 'awesome service!',
 'rocks',
 'nice work',
 "couldn't have done better"]

In [ ]:
# Se definen comentarios adicionales sobre los que se requiere predecir
new_reviews  = ["Nice!",
               "rude service",
               "terrible food"]

# Se convierten a una secuencias
sequences = tokenizer.texts_to_sequences(new_reviews)

# Se rellenan las secuencias
new_padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type)

# Se predice el sentimiento
print(model.predict(new_padded), '\n', '----'*3)
print(np.where(model.predict(new_padded)>0.5, 1, 0))

1/1 [==============================] - 0s 71ms/step
[[0.6215997 ]
 [0.41907522]
 [0.42697784]] 
 ------------
1/1 [==============================] - 0s 21ms/step
[[1]
 [0]
 [0]]


#### RNN

In [ ]:
from tensorflow.keras.layers import SimpleRNN

In [ ]:
model_rnn = Sequential()
model_rnn.add(embedding_layer)
model_rnn.add(SimpleRNN(units=64))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              152       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                4672      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,889
Trainable params: 4,889
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_rnn.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.7597 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 0.7210 - accuracy: 0.1000
Epoch 3/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6844 - accuracy: 0.7000
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6494 - accuracy: 0.9000
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6156 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 0.5828 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 15ms/step - loss: 0.5506 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 0.5190 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 0.4877 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 18ms/step - loss: 0.4566 - accuracy: 1.0000
Epoch 11/100
1/1 

In [ ]:
print(model_rnn.predict(new_padded), '\n', '----'*3)
print(np.where(model_rnn.predict(new_padded)>0.5, 1, 0))

1/1 [==============================] - 0s 137ms/step
[[0.99892765]
 [0.00210364]
 [0.1043565 ]] 
 ------------
1/1 [==============================] - 0s 21ms/step
[[1]
 [0]
 [0]]


#### GRU

In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
model_gru = Sequential()
model_gru.add(embedding_layer)
model_gru.add(GRU(units=64))
model_gru.add(Dense(1, activation='sigmoid'))

model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_gru.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              152       
                                                                 
 gru (GRU)                   (None, 64)                14208     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 14,425
Trainable params: 14,425
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_gru.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.6919 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6902 - accuracy: 0.6000
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6885 - accuracy: 0.8000
Epoch 4/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6868 - accuracy: 0.9000
Epoch 5/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6851 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6833 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6814 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6795 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6775 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6753 - accuracy: 1.0000
Epoch 11/100
1/1 [===

In [ ]:
print(model_gru.predict(new_padded), '\n', '----'*3)
print(np.where(model_gru.predict(new_padded)>0.5, 1, 0))

1/1 [==============================] - 0s 315ms/step
[[0.9994923 ]
 [0.00142507]
 [0.04930305]] 
 ------------
1/1 [==============================] - 0s 23ms/step
[[1]
 [0]
 [0]]


#### LSTM

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(embedding_layer)
model_lstm.add(LSTM(units=64))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_lstm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              152       
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 18,905
Trainable params: 18,905
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_lstm.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.6952 - accuracy: 0.1000
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6922 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6892 - accuracy: 0.7000
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6863 - accuracy: 0.9000
Epoch 5/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6833 - accuracy: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6803 - accuracy: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6772 - accuracy: 0.9000
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6740 - accuracy: 0.9000
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6708 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6673 - accuracy: 1.0000
Epoch 11/100
1/1 [===

In [ ]:
print(model_lstm.predict(new_padded), '\n', '----'*3)
print(np.where(model_lstm.predict(new_padded)>0.5, 1, 0))

1/1 [==============================] - 0s 358ms/step
[[0.997166  ]
 [0.00397032]
 [0.04999427]] 
 ------------
1/1 [==============================] - 0s 22ms/step
[[1]
 [0]
 [0]]


#### Bidireccional

In [ ]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
model_bd = Sequential()
model_bd.add(embedding_layer)
model_bd.add(Bidirectional(layer=LSTM(units=64)))
model_bd.add(Dense(1, activation='sigmoid'))

model_bd.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_bd.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              152       
                                                                 
 bidirectional (Bidirectiona  (None, 128)              37376     
 l)                                                              
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 37,657
Trainable params: 37,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_bd.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.6954 - accuracy: 0.3000
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6876 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6799 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6722 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6646 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 18ms/step - loss: 0.6569 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6492 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6414 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6335 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6255 - accuracy: 1.0000
Epoch 11/100
1/1 [===

In [ ]:
print(model_bd.predict(new_padded), '\n', '----'*3)
print(np.where(model_bd.predict(new_padded)>0.5, 1, 0))

1/1 [==============================] - 1s 725ms/step
[[0.998587  ]
 [0.00482868]
 [0.05373497]] 
 ------------
1/1 [==============================] - 0s 20ms/step
[[1]
 [0]
 [0]]


#### Convolución 1D

In [ ]:
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D

[Global Average Pooling](https://paperswithcode.com/method/global-average-pooling)

In [ ]:
model_conv = Sequential()
model_conv.add(embedding_layer)
model_conv.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
model_conv.add(GlobalAveragePooling1D())
model_conv.add(Dense(1, activation='sigmoid'))

model_conv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_conv.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              152       
                                                                 
 conv1d (Conv1D)             (None, 2, 8)              200       
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_6 (Dense)             (None, 1)                 9         
                                                                 
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_conv.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 0.6523 - accuracy: 0.7000
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 0.6466 - accuracy: 0.8000
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6410 - accuracy: 0.9000
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6355 - accuracy: 0.9000
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6300 - accuracy: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6246 - accuracy: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6192 - accuracy: 0.9000
Epoch 8/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6140 - accuracy: 0.9000
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6089 - accuracy: 0.9000
Epoch 10/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6040 - accuracy: 0.9000
Epoch 11/100
1/1 [===

In [ ]:
print(model_conv.predict(new_padded), '\n', '----'*3)
print(np.where(model_conv.predict(new_padded)>0.5, 1, 0))

1/1 [==============================] - 0s 117ms/step
[[0.7681727 ]
 [0.36779055]
 [0.26061252]] 
 ------------
1/1 [==============================] - 0s 29ms/step
[[1]
 [0]
 [0]]
